In [1]:
! pip install transformers torch pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
from huggingface_hub import login
from transformers import pipeline
import pandas as pd

# Enter your Hugging Face token
HUGGINGFACE_TOKEN = "ADD TOKEN HERE"

# Log in to Hugging Face
login(token=HUGGINGFACE_TOKEN)

In [7]:
# Model name
MODEL_NAME = "charlie1898/gpt2_finetuned_twitter_sentiment_analysis"

# Load GPT-2 tokenizer instead
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# Create sentiment analysis pipeline
sentiment_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer)

print("Model and tokenizer loaded successfully!")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


Model and tokenizer loaded successfully!


In [11]:
# ✅ Step 3: Test on Sample Comments
sample_comments = [
    "I love this video! It’s so informative.",
    "This is the worst content I've ever seen.",
    "The video is okay, not great but not terrible."
]

# Get sentiment predictions
results = sentiment_pipeline(sample_comments)
print(results)

[{'label': 'LABEL_2', 'score': 0.9986668825149536}, {'label': 'LABEL_0', 'score': 0.9993966817855835}, {'label': 'LABEL_0', 'score': 0.9840503334999084}]


In [17]:
import numpy as np

# Label mapping for the model outputs
label_mapping = {
    "LABEL_0": "Negative",
    "LABEL_1": "Neutral",
    "LABEL_2": "Positive"
}

def classify_sentiment(result):
    """ Convert model output to human-readable sentiment labels. """
    label = result["label"]
    score = result["score"]

    # If classified as Negative but confidence is low, consider it Neutral
    if label == "LABEL_0" and score < 0.99:
        return "Neutral"

    return label_mapping.get(label, "Unknown")  # Map label to readable format

# Apply to model results
final_results = [classify_sentiment(result) for result in results]

# Print the results
for comment, sentiment in zip(sample_comments, final_results):
    print(f"Comment: {comment}\nSentiment: {sentiment}\n")

Comment: I love this video! It’s so informative.
Sentiment: Positive

Comment: This is the worst content I've ever seen.
Sentiment: Negative

Comment: The video is okay, not great but not terrible.
Sentiment: Neutral



In [18]:
# ✅ Step 2: Load the CSV File
FILE_PATH = "/content/drive/MyDrive/final_project/youtube_comments.csv"  # Replace with your actual file path

try:
    df = pd.read_csv(FILE_PATH)
    print(f"✅ CSV file '{FILE_PATH}' loaded successfully! Total comments: {len(df)}\n")
except Exception as e:
    print(f"❌ Error loading CSV file: {e}")
    raise

✅ CSV file '/content/drive/MyDrive/final_project/youtube_comments.csv' loaded successfully! Total comments: 12561



In [20]:
# Ensure the correct column name (update if different)
if "text" not in df.columns:
    print("❌ Error: The column 'comment' is missing in the CSV. Check column names!")
    print("Available columns:", df.columns)
    raise ValueError("Column 'comment' not found in CSV!")

In [22]:
# Extract comments
comments = df["text"].astype(str).tolist()  # Ensure all are strings

In [23]:

# ✅ Step 3: Define Sentiment Classification Function
label_mapping = {
    "LABEL_0": "Negative",
    "LABEL_1": "Neutral",
    "LABEL_2": "Positive"
}

def classify_sentiment(result):
    """ Convert model output to human-readable sentiment labels. """
    label = result["label"]
    score = result["score"]

    # Adjust classification threshold
    if label == "LABEL_0" and score < 0.99:
        return "Neutral"

    return label_mapping.get(label, "Unknown")

In [24]:
# ✅ Step 4: Process and Save Sentiments in Real Time
output_file = "sentiment_results.csv"

In [25]:
# Open CSV for real-time writing
with open(output_file, "w") as f:
    f.write("comment,sentiment\n")  # Write header

    for idx, comment in enumerate(comments):
        # Get sentiment prediction
        result = sentiment_pipeline(comment)[0]  # Model returns a list, get first item
        sentiment = classify_sentiment(result)

        # Print in Colab
        print(f"📝 Processing {idx+1}/{len(comments)}: {comment}")
        print(f"💬 Sentiment: {sentiment}\n")

        # Write to CSV in real-time
        f.write(f'"{comment}","{sentiment}"\n')

print(f"\n✅ Sentiment analysis completed! Results saved in '{output_file}'.")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Streaming output truncated to the last 5000 lines.

📝 Processing 10961/12561: After some days, Priyanka ji will come.
💬 Sentiment: Neutral

📝 Processing 10962/12561: Jai Congress
💬 Sentiment: Neutral

📝 Processing 10963/12561: Well done Rahul.
💬 Sentiment: Positive

📝 Processing 10964/12561: Rahul Gandhi is true Hindu
💬 Sentiment: Neutral

📝 Processing 10965/12561: This is the beauty of democracy.
💬 Sentiment: Positive

📝 Processing 10966/12561: Na rahul na modi ab Rahul Kumar Modi
💬 Sentiment: Neutral

📝 Processing 10967/12561: Salute Gandhi jee. Great issue.
💬 Sentiment: Positive

📝 Processing 10968/12561: Rahul Gandhi is true Hindu
💬 Sentiment: Neutral

📝 Processing 10969/12561: Now Modiji comes to know that this is not a pappu voice ...this is  "THE PEOPLE VOICE"
💬 Sentiment: Neutral

📝 Processing 10970/12561: Modi and BJP are facing true power of oppositions now.
💬 Sentiment: Neutral

📝 Processing 10971/12561: Jab opposition bolta hai to om birla ka face dekhne layak honjata hai.
